1️⃣ Secrets & API-key bootstrap

In [ ]:
# ────────────────────────────────────────────────────────────────────────
#  SECRETS → LOCAL FILES  |  runs before any other import
# ────────────────────────────────────────────────────────────────────────
import os, base64, pathlib, json
import openai
from openai import OpenAI

def _decode_and_validate(secret_name: str, out_file: str):
    """Decode Base64 GitHub secret and validate JSON structure."""
    try:
        decoded = base64.b64decode(os.environ[secret_name]).decode('utf-8')
        data = json.loads(decoded)

        # For token.json, verify required fields
        if out_file == "token.json":
            required_fields = {'token', 'refresh_token', 'scopes'}
            if not required_fields.issubset(data.keys()):
                missing = required_fields - set(data.keys())
                raise ValueError(f"Missing required fields in token: {missing}")

        pathlib.Path(out_file).write_text(decoded)
        return True
    except Exception as e:
        print(f"❌ Error processing {secret_name}: {str(e)}")
        raise

try:
    _decode_and_validate("CLIENT_SECRETS_JSON", "client_secrets.json")
    _decode_and_validate("TOKEN_JSON", "token.json")
except Exception as e:
    print("❌ Failed to initialize secrets")
    raise

openai.api_key = os.environ["OPENAI_API_KEY"]
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]  # <-- add this line
client = OpenAI()
print("✅ Secrets decoded and validated, OpenAI client ready.")


2️⃣ Parameter cell

In [ ]:
# PARAMETERS  (Papermill / run-notebook overwrites these at run-time)
GAMEPLAY_URL = None                         # GitHub secret injects real link
TITLE_TEXT   = "Daily DALLE Short"          # you can override this too
OPENAI_MODEL = "gpt-4o"


🔐 3. Authenticate with YouTube

In [ ]:
# 🔐 2. Authenticate with YouTube (automatic – no browser needed)
from google.oauth2.credentials import Credentials
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import json

# Use only the essential scope needed for uploads
SCOPE = "https://www.googleapis.com/auth/youtube.upload"

def get_authenticated_service():
    """Create authenticated YouTube client with scope validation."""
    try:
        # Load token data
        with open("token.json") as f:
            token_data = json.load(f)

        # Verify the token has our required scope
        if 'scopes' not in token_data or SCOPE not in token_data['scopes']:
            raise ValueError(f"Token missing required scope: {SCOPE}")

        creds = Credentials.from_authorized_user_info(token_data, [SCOPE])

        # Refresh token if needed
        if creds and creds.expired and creds.refresh_token:
            try:
                creds.refresh(Request())
                # Update token file with refreshed credentials
                with open("token.json", "w") as f:
                    json.dump(json.loads(creds.to_json()), f)
            except Exception as refresh_error:
                print(f"⚠️ Token refresh failed: {refresh_error}")
                # Continue with expired token if we have one
                if not creds.token:
                    raise

        return build("youtube", "v3", credentials=creds)
    except HttpError as e:
        print(f"❌ YouTube API error: {e}")
        raise
    except Exception as e:
        print(f"❌ Authentication failed: {e}")
        raise

try:
    youtube = get_authenticated_service()
    print(f"✅ YouTube API authenticated with scope: {SCOPE}")
except Exception as e:
    print("❌ Failed to initialize YouTube client")
    raise

4-7: THIS IS WHERE WE BUILD OUR SHORTS DRAMA STORIES GENERATION CODE BLOCKS

1️⃣ Generate a Unique Story Seed (Topic)

In [ ]:
# ───────────────────────────────────────────────────────────────
# 1️⃣ Generate a unique seed/topic for each short story run

import random

def generate_story_seed():
    # You can replace/expand this later with API calls or trending topic scrapers!
    seed_topics = [
        "A young girl discovers a mysterious letter in her attic.",
        "Two strangers meet on a train and share a secret.",
        "A lost dog finds its way back home against all odds.",
        "An old man recalls the day he saved a child's life.",
        "A chance encounter changes the fate of an entire village.",
        "A child finds an ancient artifact in the woods.",
        "A teacher faces a dilemma after discovering a student's secret.",
        "A musician hears a melody that no one else can.",
        "A family reunites after years apart due to an unexpected event.",
        "A janitor uncovers a hidden talent during a school talent show.",
        "A shopkeeper helps a mysterious customer late at night."
    ]
    seed = random.choice(seed_topics)
    print(f"Selected story seed: {seed}")
    return seed

STORY_SEED = generate_story_seed()


2️⃣ Generate a Short Story Script Using GPT-4o-mini

In [ ]:
# ───────────────────────────────────────────────────────────────
# 2️⃣ Generate a short story script (30-40 seconds) with GPT-4o-mini (NEW API)

from openai import OpenAI

def generate_short_story(seed, example_story=None):
    system_prompt = (
        "You are a creative short story writer. "
        "Write a dramatic, emotional story suitable for a 30-40 second narration. "
        "Your writing should be concise, with vivid imagery and a clear buildup and resolution. "
        "Use simple language and dialogue where appropriate."
    )
    few_shot_example = (
        example_story or
    """
A boy was chased by the police. He ran very quickly into his house. His older brother saw him—his shirt and hands stained with blood. He was shocked. His brother was shaking in fear and could not speak.

The older brother looked down from the window and saw a fleet of police cars, their sirens blaring.

"We're giving you five minutes to surrender!" said a voice.

He told his younger brother, "Give me your shirt. I need to wash off the bloodstains."

A few minutes later, the younger brother heard their back door slam. He looked and saw his older brother walking toward the police, wearing his shirt with hands in the air.

He screamed with tears, "What are you doing? No, Brother, no!"

They took him away.

Days passed. It was his trial. His younger brother sat in the courtroom, tear-filled eyes watching his older brother being sentenced for life for a crime he knew nothing about.

He couldn’t hold himself back. He ran to the judge and screamed, "It was me! Please, leave my brother—he’s innocent!" Tears streamed from his eyes.

Then, suddenly, the police rushed in and showed CCTV footage proving the boy’s innocence. The real culprit had been caught.

"Why did you do that?" he asked his older brother.

He replied, "I am your big brother. I’d give my life for you."
    """
    )

    prompt = (
        f"{system_prompt}\n"
        f"Example structure:\n{few_shot_example}\n\n"
        f"Now, write a unique story based on this topic:\n\"{seed}\""
    )

    # Use the OpenAI client instance, with your API key already set via environment/secrets
    client = OpenAI()  # This picks up your API key from the env

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt}
        ],
        max_tokens=600,
        temperature=1.0,
    )
    story_script = response.choices[0].message.content.strip()
    print(f"Generated short story:\n{story_script}")
    return story_script

STORY_SCRIPT = generate_short_story(STORY_SEED)


3️⃣ Generate Relevant Images with DALL·E 3 (Segmented for Story)

In [ ]:
# ───────────────────────────────────────────────────────────────
# 3️⃣ Generate relevant images for story segments using DALL·E 3 (NEW API + GPT prompt shortener)

import math
from PIL import Image, ImageOps
import requests
from openai import OpenAI
import re

def pad_to_vertical(img_path, target_size=(1080, 1920), fill_color=(0,0,0)):
    img = Image.open(img_path)
    img = ImageOps.contain(img, (target_size[0], target_size[1]), method=Image.LANCZOS)
    pad_img = Image.new("RGB", target_size, fill_color)
    offset_x = (target_size[0] - img.width) // 2
    offset_y = (target_size[1] - img.height) // 2
    pad_img.paste(img, (offset_x, offset_y))
    pad_img.save(img_path)
    return img_path

def split_script_for_images(script, num_images=1):
    # Split by sentence, but group if short.
    sentences = re.split(r'(?<=[.!?]) +', script)
    avg = math.ceil(len(sentences) / num_images)
    segments = [' '.join(sentences[i:i+avg]) for i in range(0, len(sentences), avg)]
    if len(segments) > num_images:
        segments = segments[:num_images]
    return segments

def get_visual_prompt(client, chunk):
    """Use GPT-4o-mini to generate a short, DALL·E-3-safe prompt."""
    prompt = (
        "Summarize the following story segment into a short, visually descriptive DALL·E-3 prompt. "
        "Avoid names, violence, or explicit actions. Focus on the main scene, setting, and mood, under 15 words:\n"
        f"{chunk}"
    )
    resp = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are an expert at generating concise, visual art prompts."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=50,
        temperature=0.4,
    )
    short_prompt = resp.choices[0].message.content.strip()
    short_prompt = short_prompt.strip('"')
    return short_prompt

def generate_dalle3_images(story_script, num_images=6):
    client = OpenAI()
    segments = split_script_for_images(story_script, num_images)
    image_files = []
    for i, segment in enumerate(segments):
        visual_prompt = get_visual_prompt(client, segment)
        dalle_prompt = f"Digital art, cinematic style, {visual_prompt}"
        print(f"🖼️ Prompt for DALL·E image {i+1}: {dalle_prompt}")
        try:
            response = client.images.generate(
                model="dall-e-3",
                prompt=dalle_prompt,
                n=1,
                size="1024x1024",
                quality="standard",
                response_format="url"
            )
            img_url = response.data[0].url
            img_data = requests.get(img_url).content
            img_path = f"story_img_{i+1}.png"
            with open(img_path, "wb") as f:
                f.write(img_data)
            pad_to_vertical(img_path)
            image_files.append(img_path)
            print(f"Generated image for segment {i+1}: {img_path}")
        except Exception as e:
            print(f"❌ DALL·E image generation failed for segment {i+1}: {e}")
            continue  # Optionally: skip or use placeholder
    return image_files, segments

IMAGE_FILES, STORY_SEGMENTS = generate_dalle3_images(STORY_SCRIPT, num_images=6)


4️⃣ Generate Narration with OpenAI TTS (Echo Voice)

In [ ]:
# ───────────────────────────────────────────────────────────────
# 4️⃣ Generate TTS narration using OpenAI tts-1 (echo voice, NEW API)

from openai import OpenAI

def generate_tts_narration(story_script, output_path="narration.mp3"):
    client = OpenAI()  # Uses API key from environment
    tts_response = client.audio.speech.create(
        model="tts-1",
        voice="echo",
        input=story_script,
        response_format="mp3"
    )
    tts_response.stream_to_file(output_path)
    print(f"Narration audio saved to {output_path}")
    return output_path

NARRATION_AUDIO_PATH = generate_tts_narration(STORY_SCRIPT)


5️⃣ Generate Synced Subtitles (GPT for Chunks Matching Narration Timing)

In [ ]:
# ───────────────────────────────────────────────────────────────
# 5️⃣ Generate subtitle chunks using GPT and sync to narration (NEW API)

from openai import OpenAI

def chunk_script_for_subtitles(story_script, narration_path, max_words=8):
    client = OpenAI()  # Uses your env API key

    # First, split script into small chunks using GPT
    chunk_prompt = (
        "Split the following story into short, natural subtitle lines, "
        f"each with no more than {max_words} words. "
        "Return the result as a Python list of strings."
    )
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": f"{chunk_prompt}\n\nStory:\n{story_script}"}
        ],
        max_tokens=512,
        temperature=0.2,
    )
    import ast
    raw_chunks = response.choices[0].message.content.strip()
    try:
        subtitle_chunks = ast.literal_eval(raw_chunks)
    except Exception:
        # fallback: naive split
        subtitle_chunks = story_script.split('. ')
    print("Subtitle chunks generated:", subtitle_chunks)
    # Now, get narration duration and assign times
    from mutagen.mp3 import MP3
    audio = MP3(narration_path)
    total_duration = audio.info.length
    per_chunk = total_duration / len(subtitle_chunks)
    subtitle_timings = []
    for idx, chunk in enumerate(subtitle_chunks):
        start = idx * per_chunk
        end = start + per_chunk
        subtitle_timings.append({"text": chunk, "start": start, "end": end})
    return subtitle_timings

# Install mutagen for MP3 duration reading if needed
!pip install mutagen --quiet

SUBTITLES = chunk_script_for_subtitles(STORY_SCRIPT, NARRATION_AUDIO_PATH)


6️⃣ Combine All Into Shorts Video (Images + Narration + Subtitles)

In [ ]:
# ───────────────────────────────────────────────────────────────
# Font & Size setup (insert above video function, can go right at the top of this cell)
import sys

if sys.platform.startswith("linux"):
    TEXT_FONT_PATH = "/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf"
elif sys.platform.startswith("win"):
    TEXT_FONT_PATH = "C:/Windows/Fonts/DejaVuSans-Bold.ttf"
else:
    TEXT_FONT_PATH = "DejaVuSans-Bold.ttf"
TEXT_FONTSIZE = 60

# Robust timeline and resize helpers for MoviePy 1.x and 2.x
def _set_duration(clip, duration):
    if hasattr(clip, "with_duration"): return clip.with_duration(duration)
    return clip.set_duration(duration)

def _set_audio(clip, audio):
    if hasattr(clip, "with_audio"): return clip.with_audio(audio)
    return clip.set_audio(audio)

def _set_position(clip, pos):
    if hasattr(clip, "with_position"): return clip.with_position(pos)
    return clip.set_position(pos)

def _set_start(clip, start):
    if hasattr(clip, "with_start"): return clip.with_start(start)
    return clip.set_start(start)

def resize_fx(clip, newsize):
    # newsize: (width, height)
    if hasattr(clip, "resized"):
        return clip.resized(newsize)
    return clip.resize(newsize)

# --- Robust imports ---
from moviepy import ImageClip, TextClip, CompositeVideoClip, concatenate_videoclips, AudioFileClip

def create_shorts_video(image_files, narration_path, subtitles, output_path="shorts_final.mp4"):
    audio = AudioFileClip(narration_path)
    total_duration = audio.duration
    num_images = len(image_files)
    img_duration = total_duration / num_images

    # Prepare image clips (robust resize)
    clips = []
    for idx, img in enumerate(image_files):
        base_clip = ImageClip(img)
        base_clip = resize_fx(base_clip, (1080, 1920))
        base_clip = _set_duration(base_clip, img_duration)
        base_clip = _set_position(base_clip, 'center')
        clips.append(base_clip)
    video = concatenate_videoclips(clips, method="compose")
    video = _set_audio(video, audio)

    # Prepare subtitle clips (fade in/out for nicer effect)
    subtitle_clips = []
    for sub in subtitles:
        txt = TextClip(
            sub["text"],
            fontsize=TEXT_FONTSIZE,
            color='white',
            bg_color="black",
            method='caption',
            size=(1000, None),
            align='center'
        )
        # Note: Removed 'font=TEXT_FONT_PATH' for 'caption' mode compatibility!
        txt = _set_start(txt, sub["start"])
        txt = _set_duration(txt, sub["end"] - sub["start"])
        txt = _set_position(txt, ("center", "bottom")).margin(bottom=60, opacity=0)
        subtitle_clips.append(txt)

    # Overlay subtitles
    final = CompositeVideoClip([video] + subtitle_clips)
    final.write_videofile(output_path, fps=24, codec="libx264", audio_codec="aac")
    print(f"Final shorts video saved to {output_path}")
    return output_path

SHORTS_VIDEO_PATH = create_shorts_video(IMAGE_FILES, NARRATION_AUDIO_PATH, SUBTITLES)


#@📝 8. Generate Metadata

In [ ]:
# ─── 8. Generate Metadata (title ≠ background_topic) ───────────────────────
def generate_metadata(topic):
    prompt = (
        f"Generate a YouTube Short title about '{topic}' that follows these rules:\n"
        "1. Use emotional triggers (shock, curiosity, awe) and keep under 40 characters\n"
        "2. Include numbers/superlatives when possible without being clickbaity\n"
        "3. Don't exactly match the topic but remain highly relevant\n\n"
        "Then provide a description that:\n"
        "1. Starts with the most important keyword\n"
        "2. Includes a CTA (Like/Follow/Comment)\n"
        "3. Is under 100 words with 2-3 hashtags in first line\n\n"
        "Finally provide 20-30 specific SEO tags (comma-separated) including:\n"
        "1. 2-3 exact match keyword phrases\n"
        "2. 2-3 related niche terms\n"
        "3. These required tags: #shorts #short #youtubeshorts #viralshorts #shortsfeed"
    )

    resp = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role":"user","content":prompt}],
        temperature=0.85,  # Slightly more creative
        max_tokens=300,    # Allow more length
        presence_penalty=0.5  # Encourages diverse terms
    )
    text = resp.choices[0].message.content.strip()

    # Parse metadata (keeping same structure)
    title, desc, tags = None, None, []
    for line in text.splitlines():
        low = line.lower()
        if low.startswith("title:"):
            title = line.split(":",1)[1].strip()
            # Remove surrounding quotes if present
            title = title.strip(' "\'')
            # Add engagement punctuation if missing
            if title and not title[-1] in ('!', '?', '.'):
                title += '!'
        if low.startswith("description:"):
            desc = line.split(":",1)[1].strip()
            # Enhance description format if exists
            if desc:
                desc = f"{desc}\n\n🔥 Like for more!\n💬 Comment your thoughts!"
                desc = desc[:500]  # Ensure under character limit
        if low.startswith("tags:"):
            tags = [t.strip() for t in line.split(":",1)[1].split(",")]

    # Ensure Shorts requirements (maintaining same checks)
    tags = [tag for tag in tags if tag]  # Remove empty tags
    required_tags = ["#shorts", "#short","#Shortsviral", "fyp", "viralshorts", "viralnews", "globalnews", "#trending", "#youtubeshorts"]
    for req_tag in required_tags:
        if req_tag not in [t.lower() for t in tags]:
            tags.append(req_tag)
    # Clean tags
    tags = [tag[:25] for tag in tags if tag]  # Length limit and remove empties
    tags = list(set(tags))[:30]  # Deduplicate and limit to 30 tags

    return title, desc, tags

title, description, tags = generate_metadata(background_topic)
print("📝 Metadata:\n ", title, "\n ", description, "\n ", tags)


📤 9. Upload to YouTube

In [ ]:
# ─── 9. Upload to YouTube ───────────────────────────────────────────────
from googleapiclient.http import MediaFileUpload
from googleapiclient.errors import HttpError
import time, os

def upload_video_to_youtube(
    file_path, title, description, tags,
    category_id=22, privacy="public",
    chunk_size=1024*1024, max_retries=5
):
    # Enforce Shorts requirements
    if not any(t.lower() == "#shorts" for t in tags):
        tags.append("#shorts")

    body = {
        "snippet": {
            "title": title,  # Removed #shorts append
            "description": f"{description}\n\n#shorts",  # Kept in description
            "tags": tags,
            "categoryId": str(category_id)
        },
        "status": {
            "privacyStatus": privacy,
            "selfDeclaredMadeForKids": False
        },
        "contentDetails": {
            "duration": "PT60S"  # Force recognition as Short
        }
    }

    media = MediaFileUpload(
        file_path,
        mimetype='video/mp4',
        chunksize=chunk_size,
        resumable=True
    )

    req = youtube.videos().insert(
        part="snippet,status,contentDetails",
        body=body,
        media_body=media
    )

    done = False
    retry = 0
    while not done:
        try:
            status, resp = req.next_chunk()
            if status:
                print(f"🟢 {int(status.progress()*100)}% uploaded")
            else:
                done = True
        except HttpError as e:
            if e.resp.status in [500,502,503,504] and retry < max_retries:
                retry += 1
                time.sleep(2**retry)
                print(f"⚠️ Retry {retry}")
            else:
                raise

    print("✅ Upload complete! Video ID:", resp["id"])
    return resp

response = upload_video_to_youtube("shorts_final.mp4", title, description, tags)
print("🎉 Done! https://youtu.be/" + response["id"])